In [1]:
%matplotlib inline
import pandas as pd, datetime as dt
import collections
import numpy as np
from sklearn import datasets, linear_model
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import glob, os

from pandas import ExcelWriter

### Empty DataFrame with proper index and headers

In [7]:
arrays = [np.array(['fuel','TA_CO2','NOx limit','N_CD_23','N_CD_23','N_CD_23','N-10%_CD_23','N-10%_CD_23','N-10%_CD_23',
                    'N+10%_CD_23', 'N+10%_CD_23','N+10%_CD_23','N_CD_10','N_CD_10','N_CD_10',
                   'N_CD_30','N_CD_30','N_CD_30','N_HT_23','N_HT_23','N_HT_23',
                   'N_HT_AC18+L_23','N_HT_AC18+L_23','N_HT_AC18+L_23','W_CD_23','W_CD_23','W_CD_23',
                   'W_HT_23','W_HT_23','W_HT_23','W_CD-7','W_CD-7','W_CD-7','W_HT-7','W_HT-7','W_HT-7',
                   'ESP','ESP','ESP','ESD','ESD','ESD','LAB','LAB','LAB','LAD','LAD','LAD',
                   'MIL','MIL','MIL','SAC','SAC','SAC']),
          np.array(['','','','CO2', 'NOx', 'PN', 'CO2', 'NOx', 'PN', 'CO2', 'NOx', 'PN','CO2', 'NOx', 'PN',
                   'CO2', 'NOx', 'PN', 'CO2', 'NOx', 'PN', 'CO2', 'NOx', 'PN','CO2', 'NOx', 'PN',
                   'CO2', 'NOx', 'PN', 'CO2', 'NOx', 'PN', 'CO2', 'NOx', 'PN',
                   'CO2', 'NOx', 'PN','CO2', 'NOx', 'PN','CO2', 'NOx', 'PN','CO2', 'NOx', 'PN',
                   'CO2', 'NOx', 'PN','CO2', 'NOx', 'PN']),
          np.array(['','[g/km]','[mg/km]','[g/km]', '[mg/km]', '[#/km]', '[g/km]', '[mg/km]', '[#/km]', '[g/km]',
                    '[g/km]', '[mg/km]', '[#/km]', '[g/km]', '[mg/km]', '[#/km]', '[g/km]', '[mg/km]', '[#/km]',
                    '[mg/km]', '[#/km]','[g/km]', '[mg/km]', '[#/km]', '[g/km]', '[mg/km]', '[#/km]',
                    '[g/km]', '[mg/km]', '[#/km]', '[g/km]', '[mg/km]', '[#/km]', '[g/km]', '[mg/km]', '[#/km]',
                    '[g/km]', '[mg/km]', '[#/km]', '[g/km]', '[mg/km]', '[#/km]', '[g/km]', '[mg/km]', '[#/km]',
                    '[g/km]', '[mg/km]', '[#/km]', '[g/km]', '[mg/km]', '[#/km]', '[g/km]', '[mg/km]', '[#/km]'])]
tuples = list(zip(*arrays))
tuples

index = pd.MultiIndex.from_tuples(tuples, names=['test', 'pollutant','[unit]'])
index

df = pd.DataFrame(np.random.randn(11, 54),
                  index=['Audi A1', 'BMW 530d', 'Citroen Cactus','Fiat 500x','Opel Astra',
                         'Fiat Panda','Porsche Cayenne','VW Golf','VW Tiguan','Audi A3','Peugeot Partner'], columns=index)


# df = pd.DataFrame(np.random.randn(54, 11),
#                   index=index, columns=['Audi A1', 'BMW 530d', 'Citroen Cactus','Fiat 500x','Opel Astra',
#                          'Fiat Panda','Porsche Cayenne','VW Golf','VW Tiguan','Audi A3','Peugeot Partner'])

df = pd.DataFrame(columns=df.columns, index=df.index)            # this converts random numbers into NaN
df

test            fuel TA_CO2 NOx limit N_CD_23                N-10%_CD_23  \
pollutant                                 CO2     NOx     PN         CO2   
[unit]               [g/km]   [mg/km]  [g/km] [mg/km] [#/km]      [g/km]   
Audi A1          NaN    NaN       NaN     NaN     NaN    NaN         NaN   
BMW 530d         NaN    NaN       NaN     NaN     NaN    NaN         NaN   
Citroen Cactus   NaN    NaN       NaN     NaN     NaN    NaN         NaN   
Fiat 500x        NaN    NaN       NaN     NaN     NaN    NaN         NaN   
Opel Astra       NaN    NaN       NaN     NaN     NaN    NaN         NaN   
Fiat Panda       NaN    NaN       NaN     NaN     NaN    NaN         NaN   
Porsche Cayenne  NaN    NaN       NaN     NaN     NaN    NaN         NaN   
VW Golf          NaN    NaN       NaN     NaN     NaN    NaN         NaN   
VW Tiguan        NaN    NaN       NaN     NaN     NaN    NaN         NaN   
Audi A3          NaN    NaN       NaN     NaN     NaN    NaN         NaN   
Peugeot Partner  NaN    NaN       NaN     NaN     NaN    NaN         NaN   

test                           N+10%_CD_23  ...      LAB    LAD          \
pollutant           NOx     PN         CO2  ...       PN    CO2     NOx   
[unit]          [mg/km] [#/km]      [g/km]  ...   [#/km] [g/km] [mg/km]   
Audi A1             NaN    NaN         NaN  ...      NaN    NaN     NaN   
BMW 530d            NaN    NaN         NaN  ...      NaN    NaN     NaN   
Citroen Cactus      NaN    NaN         NaN  ...      NaN    NaN     NaN   
Fiat 500x           NaN    NaN         NaN  ...      NaN    NaN     NaN   
Opel Astra          NaN    NaN         NaN  ...      NaN    NaN     NaN   
Fiat Panda          NaN    NaN         NaN  ...      NaN    NaN     NaN   
Porsche Cayenne     NaN    NaN         NaN  ...      NaN    NaN     NaN   
VW Golf             NaN    NaN         NaN  ...      NaN    NaN     NaN   
VW Tiguan           NaN    NaN         NaN  ...      NaN    NaN     NaN   
Audi A3             NaN    NaN         NaN  ...      NaN    NaN     NaN   
Peugeot Partner     NaN    NaN         NaN  ...      NaN    NaN     NaN   

test                      MIL                   SAC                 
pollutant           PN    CO2     NOx     PN    CO2     NOx     PN  
[unit]          [#/km] [g/km] [mg/km] [#/km] [g/km] [mg/km] [#/km]  
Audi A1            NaN    NaN     NaN    NaN    NaN     NaN    NaN  
BMW 530d           NaN    NaN     NaN    NaN    NaN     NaN    NaN  
Citroen Cactus     NaN    NaN     NaN    NaN    NaN     NaN    NaN  
Fiat 500x          NaN    NaN     NaN    NaN    NaN     NaN    NaN  
Opel Astra         NaN    NaN     NaN    NaN    NaN     NaN    NaN  
Fiat Panda         NaN    NaN     NaN    NaN    NaN     NaN    NaN  
Porsche Cayenne    NaN    NaN     NaN    NaN    NaN     NaN    NaN  
VW Golf            NaN    NaN     NaN    NaN    NaN     NaN    NaN  
VW Tiguan          NaN    NaN     NaN    NaN    NaN     NaN    NaN  
Audi A3            NaN    NaN     NaN    NaN    NaN     NaN    NaN  
Peugeot Partner    NaN    NaN     NaN    NaN    NaN     NaN    NaN  

[11 rows x 54 columns]

In [ ]:
df = pd.DataFrame(np.random.randn(11, 54),
                  index=['Audi A1', 'BMW 530d', 'Citroen Cactus','Fiat 500x','Opel Astra',
                         'Fiat Panda','Porsche Cayenne','VW Golf','VW Tiguan','Audi A3','Peugeot Partner'], columns=index)

df = pd.DataFrame(columns=df.columns, index=df.index)            # this converts random numbers into NaN
df

### Add few columns of useful vehicle data

In [27]:
# idx = 0
# new_col = ['gasoline','diesel','diesel','diesel','gasoline',
#            'gasoline','diesel','diesel','diesel','diesel','diesel']  
# df.insert(loc=idx, column='fuel', value=new_col)

# idx = 1
# new_col = [98,124,95,144,103,119,179,117,159,120,112]  
# df.insert(loc=idx, column='TA_CO2', value=new_col)

# idx = 2
# new_col = [60,80,80,80,60,60,80,80,80,80,80]  
# df.insert(loc=idx, column='NOx_limit', value=new_col)


# df

test                 fuel TA_CO2 NOx_limit N_CD_23                N-10%_CD_23  \
pollutant                                      CO2     NOx     PN         CO2   
[unit]                                      [g/km] [mg/km] [#/km]      [g/km]   
Audi A1          gasoline     98        60     NaN     NaN    NaN         NaN   
BMW 530d           diesel    124        80     NaN     NaN    NaN         NaN   
Citroen Cactus     diesel     95        80     NaN     NaN    NaN         NaN   
Fiat 500x          diesel    144        80     NaN     NaN    NaN         NaN   
Opel Astra       gasoline    103        60     NaN     NaN    NaN         NaN   
Fiat Panda       gasoline    119        60     NaN     NaN    NaN         NaN   
Porsche Cayenne    diesel    179        80     NaN     NaN    NaN         NaN   
VW Golf            diesel    117        80     NaN     NaN    NaN         NaN   
VW Tiguan          diesel    159        80     NaN     NaN    NaN         NaN   
Audi A3            diesel    120        80     NaN     NaN    NaN         NaN   
Peugeot Partner    diesel    112        80     NaN     NaN    NaN         NaN   

test                           N+10%_CD_23  ...      LAB    LAD          \
pollutant           NOx     PN         CO2  ...       PN    CO2     NOx   
[unit]          [mg/km] [#/km]      [g/km]  ...   [#/km] [g/km] [mg/km]   
Audi A1             NaN    NaN         NaN  ...      NaN    NaN     NaN   
BMW 530d            NaN    NaN         NaN  ...      NaN    NaN     NaN   
Citroen Cactus      NaN    NaN         NaN  ...      NaN    NaN     NaN   
Fiat 500x           NaN    NaN         NaN  ...      NaN    NaN     NaN   
Opel Astra          NaN    NaN         NaN  ...      NaN    NaN     NaN   
Fiat Panda          NaN    NaN         NaN  ...      NaN    NaN     NaN   
Porsche Cayenne     NaN    NaN         NaN  ...      NaN    NaN     NaN   
VW Golf             NaN    NaN         NaN  ...      NaN    NaN     NaN   
VW Tiguan           NaN    NaN         NaN  ...      NaN    NaN     NaN   
Audi A3             NaN    NaN         NaN  ...      NaN    NaN     NaN   
Peugeot Partner     NaN    NaN         NaN  ...      NaN    NaN     NaN   

test                      MIL                   SAC                 
pollutant           PN    CO2     NOx     PN    CO2     NOx     PN  
[unit]          [#/km] [g/km] [mg/km] [#/km] [g/km] [mg/km] [#/km]  
Audi A1            NaN    NaN     NaN    NaN    NaN     NaN    NaN  
BMW 530d           NaN    NaN     NaN    NaN    NaN     NaN    NaN  
Citroen Cactus     NaN    NaN     NaN    NaN    NaN     NaN    NaN  
Fiat 500x          NaN    NaN     NaN    NaN    NaN     NaN    NaN  
Opel Astra         NaN    NaN     NaN    NaN    NaN     NaN    NaN  
Fiat Panda         NaN    NaN     NaN    NaN    NaN     NaN    NaN  
Porsche Cayenne    NaN    NaN     NaN    NaN    NaN     NaN    NaN  
VW Golf            NaN    NaN     NaN    NaN    NaN     NaN    NaN  
VW Tiguan          NaN    NaN     NaN    NaN    NaN     NaN    NaN  
Audi A3            NaN    NaN     NaN    NaN    NaN     NaN    NaN  
Peugeot Partner    NaN    NaN     NaN    NaN    NaN     NaN    NaN  

[11 rows x 54 columns]

### Save DataFrame to Excel

In [4]:
writer = ExcelWriter('vehicles.xlsx')
df.to_excel(writer,'Sheet1')

#road.to_excel(writer,'Sheet1',columns = ['CO2_COMP','NOx_COMP'])
writer.save()

### Open the Excel as a Dataframe

In [14]:
input_file = 'vehicles.xlsx'
vehicles = pd.read_excel(input_file, header=1)           # header is the ### row